<a href="https://colab.research.google.com/github/TechDailyNotes/study-notes-cuda/blob/main/cuda_parallel_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpulgv1rcd".


# Chapter 1: CUDA Parallel Reduction Part 1

In [ ]:
%%cuda

#include <assert.h>
#include <cuda_runtime.h>
#include <stdio.h>

const int GRID_DIM_X = 1 << 8;
const int BLOCK_DIM_X = 1 << 8;

__global__ void sumReduce(int *vector, int *vectorSum) {
    // Step 0: Get the current thread's index.
    int ti = blockIdx.x * blockDim.x + threadIdx.x;

    // Step 1: Move elements from memory to cache.
    __shared__ int partialSum[BLOCK_DIM_X];
    partialSum[threadIdx.x] = vector[ti];
    __syncthreads();

    // Step 2: Divide and conquer the sum in one block.
    for (int si = 1; si < BLOCK_DIM_X; si *= 2) {
        if (threadIdx.x % (si * 2) == 0) {
            partialSum[threadIdx.x] += partialSum[threadIdx.x + si];
        }
        __syncthreads();
    }

    // Step 3: Move the sum from cache to memory.
    if (threadIdx.x == 0) {
        vectorSum[blockIdx.x] = partialSum[0];
    }
}

void vectorInit(int *h_vector, int numElements) {
    for (int i = 0; i < numElements; i++) {
        h_vector[i] = 1;
    }
}

int main() {
    // Step 0: Set the number and bytes of the vector.
    int numElements = GRID_DIM_X * BLOCK_DIM_X;
    size_t numBytes = sizeof(int) * numElements;

    // Step 1: Initialize the host and device memories.
    int *h_vector = (int*) malloc(numBytes);
    int *h_vectorSum = (int*) malloc(numBytes);
    vectorInit(h_vector, numElements);

    int *d_vector, *d_vectorSum;
    cudaMalloc(&d_vector, numBytes);
    cudaMalloc(&d_vectorSum, numBytes);

    // Step 2: Launch the kernel function to sum up the vector.
    cudaMemcpy(d_vector, h_vector, numBytes, cudaMemcpyHostToDevice);
    sumReduce<<<GRID_DIM_X, BLOCK_DIM_X>>>(d_vector, d_vectorSum);
    sumReduce<<<1, BLOCK_DIM_X>>>(d_vectorSum, d_vectorSum);
    cudaMemcpy(h_vectorSum, d_vectorSum, numBytes, cudaMemcpyDeviceToHost);

    printf("h_vectorSum[0] == %d\n", h_vectorSum[0]);
    assert(h_vectorSum[0] == 65536);

    // Step 3: Clear the allocated memories.
    free(h_vector);
    free(h_vectorSum);
    cudaFree(d_vector);
    cudaFree(d_vectorSum);

    printf("Success!");
    return 0;
}

h_vectorSum[0] == 65536
Success!


## Practice

In [ ]:
%%cuda

#include <algorithm>
#include <cuda_runtime.h>
#include <stdio.h>
#include <time.h>

using namespace std;

const int GRID_DIM_X = 1 << 8;
const int BLOCK_DIM_X = 1 << 8;

__global__ void sumReduce(int *d_vector, int *d_vectorSum) {
    // Step 0: Get the thread id and element id.
    int vi = blockDim.x * blockIdx.x + threadIdx.x;
    int ti = threadIdx.x;

    // Step 1: Move elements from the vector to the cache.
    __shared__ int partialSum[BLOCK_DIM_X];
    partialSum[ti] = d_vector[vi];
    __syncthreads();

    // Step 2: Sum all elements in the same block.
    for (int si = 1; si < BLOCK_DIM_X; si *= 2) {
        if (ti % (2 * si) == 0) {
            partialSum[ti] += partialSum[ti + si];
        }
        __syncthreads();
    }

    // Step 3: Move the sum value to the vector.
    if (threadIdx.x == 0) {
        d_vectorSum[blockIdx.x] = partialSum[0];
    }
}

void vectorInit(int *h_vector, int numElements) {
    fill_n(h_vector, numElements, 1);
    // memset(h_vector, 1, numElements);
}

int main() {
    // Step 0: Set the hyperparameters of vectors.
    int numElements = GRID_DIM_X * BLOCK_DIM_X;
    size_t numBytes = sizeof(int) * numElements;

    // Step 1: Initialize memories for vectors in both the host and device.
    int *h_vector = (int*) malloc(numBytes);
    int *h_vectorSum = (int*) malloc(numBytes);
    vectorInit(h_vector, numElements);

    int *d_vector, *d_vectorSum;
    cudaMalloc(&d_vector, numBytes);
    cudaMalloc(&d_vectorSum, numBytes);

    // Step 2: Launch the kernel function to sum up all elements.
    cudaMemcpy(d_vector, h_vector, numBytes, cudaMemcpyHostToDevice);

    time_t start = time(NULL);
    sumReduce<<<GRID_DIM_X, BLOCK_DIM_X>>>(d_vector, d_vectorSum);
    sumReduce<<<1, BLOCK_DIM_X>>>(d_vectorSum, d_vectorSum);
    time_t end = time(NULL);
    printf("Time taken is %f seconds.\n", difftime(end, start));

    cudaMemcpy(h_vectorSum, d_vectorSum, numBytes, cudaMemcpyDeviceToHost);
    printf("h_vectorSum[0] == %d\n", h_vectorSum[0]);

    // Step 3: Clear allocated memories.
    free(h_vector);
    free(h_vectorSum);
    cudaFree(d_vector);
    cudaFree(d_vectorSum);

    printf("Success!");
    return 0;
}

Time taken is 0.000000 seconds.
h_vectorSum[0] == 65536
Success!


# Chapter 2: CUDA Parallel Reduction Part 2

## Optimizations

1. Get rid of the wrap divergence.
2. Get rid of the modulo operation.

In [ ]:
%%cuda

#include <algorithm>
#include <cuda_runtime.h>
#include <stdio.h>
#include <time.h>

const int GRID_DIM_X = 1 << 8;
const int BLOCK_DIM_X = 1 << 8;

__global__ void reduceSum(int *vector, int *vectorSum) {
    // Step 0: Get the vector index and the thread index.
    int ti = threadIdx.x;
    int vi = blockIdx.x * blockDim.x + threadIdx.x;

    // Step 1: Move elements from the vector to the cache.
    __shared__ int partialSum[BLOCK_DIM_X];
    partialSum[ti] = vector[vi];
    __syncthreads();

    // Step 2: Accumulate all elements.
    for (int si = 1; si < blockDim.x; si *= 2) {
        int index = 2 * si * ti;
        if (index < blockDim.x) {
            partialSum[index] += partialSum[index + si];
        }
        __syncthreads();
    }

    // Step 3: Move the sum to the vector.
    if (ti == 0) {
        vectorSum[blockIdx.x] = partialSum[0];
    }
}

int main() {
    // Step 0: Setup the parameters.
    int numElements = GRID_DIM_X * BLOCK_DIM_X;
    size_t numBytes = sizeof(int) * numElements;

    // Step 1: Initialize both the cpu and gpu memories.
    int *h_vector = (int*) malloc(numBytes);
    int *h_vectorSum = (int*) malloc(numBytes);
    std::fill_n(h_vector, numElements, 1);

    int *d_vector, *d_vectorSum;
    cudaMalloc(&d_vector, numBytes);
    cudaMalloc(&d_vectorSum, numBytes);

    // Step 2: Launch the reduce sum kernel function.
    cudaMemcpy(d_vector, h_vector, numBytes, cudaMemcpyHostToDevice);

    time_t start, end;
    time(&start);
    reduceSum<<<GRID_DIM_X, BLOCK_DIM_X>>>(d_vector, d_vectorSum);
    reduceSum<<<1, BLOCK_DIM_X>>>(d_vectorSum, d_vectorSum);
    time(&end);
    printf("Time taken is %f seconds.\n", difftime(end, start));

    cudaMemcpy(h_vectorSum, d_vectorSum, numBytes, cudaMemcpyDeviceToHost);
    printf("Accumulated result is %d.\n", h_vectorSum[0]);

    // Step 3: Clear allocated memories.
    free(h_vector);
    free(h_vectorSum);
    cudaFree(d_vector);
    cudaFree(d_vectorSum);

    printf("Success!");
    return 0;
}

Time taken is 0.000000 seconds.
Accumulated result is 65536.
Success!


# Chapter 3: CUDA Parallel Reduction Part 3

## Optimizations

1. Get rid of the bank conflict.

In [ ]:
%%cuda

#include <algorithm>
#include <assert.h>
#include <cuda_runtime.h>
#include <stdio.h>

const int GRID_DIM_X = 1 << 8;
const int BLOCK_DIM_X = 1 << 8;

__global__ void reduceSum(int *vector, int *vectorSum) {
    // Step 0: Get the vector element index and the thread index.
    int ti = threadIdx.x;
    int vi = blockIdx.x * blockDim.x + threadIdx.x;

    // Step 1: Move vector elements to cache.
    __shared__ int shared[BLOCK_DIM_X];
    shared[ti] = vector[vi];
    __syncthreads();

    // Step 2: Accumulate elements in the same thread block.
    for (int si = BLOCK_DIM_X / 2; si > 0; si >>= 1) {
        if (ti < si) {
            shared[ti] += shared[ti + si];
        }
        __syncthreads();
    }

    // Step 3: Move result from cache to vector.
    if (ti == 0) {
        vectorSum[blockIdx.x] = shared[0];
    }
}

int main() {
    // Step 0: Set up hyperparameters of the vector.
    int numElements = GRID_DIM_X * BLOCK_DIM_X;
    size_t numBytes = sizeof(int) * numElements;

    // Step 1: Init vector memories on both the cpu and the gpu.
    int *h_vector = (int*) malloc(numBytes);
    int *h_vectorSum = (int*) malloc(numBytes);
    std::fill_n(h_vector, numElements, 1);

    int *d_vector, *d_vectorSum;
    cudaMalloc(&d_vector, numBytes);
    cudaMalloc(&d_vectorSum, numBytes);

    // Step 2: Launch the kernel function to sum up the vector.
    cudaMemcpy(d_vector, h_vector, numBytes, cudaMemcpyHostToDevice);
    reduceSum<<<GRID_DIM_X, BLOCK_DIM_X>>>(d_vector, d_vectorSum);
    reduceSum<<<1, BLOCK_DIM_X>>>(d_vectorSum, d_vectorSum);
    cudaMemcpy(h_vectorSum, d_vectorSum, numBytes, cudaMemcpyDeviceToHost);
    printf("Accumulated result is %d.\n", h_vectorSum[0]);
    assert(h_vectorSum[0] == 65536);

    // Step 3: Clear vector memories.
    free(h_vector);
    free(h_vectorSum);
    cudaFree(d_vector);
    cudaFree(d_vectorSum);

    printf("Success!");
    return 0;
}

Accumulated result is 65536.
Success!


# Chapter 4: CUDA Parallel Reduction Part 4

## Optimizations

1. Get rid of idle threads.

In [ ]:
%%cuda

#include <algorithm>
#include <assert.h>
#include <cuda_runtime.h>
#include <stdio.h>

const int FACTOR = 2;
const int GRID_DIM_X = (1 << 8) / FACTOR;
const int BLOCK_DIM_X = 1 << 8;

__global__ void reduceSum(int *vector, int *vectorSum) {
    // Step 0: Get the vector element index and the thread index.
    int ti = threadIdx.x;
    int vi = blockIdx.x * (blockDim.x * 2) + threadIdx.x;

    // Step 1: Move vector elements to cache.
    __shared__ int shared[BLOCK_DIM_X];
    shared[ti] = vector[vi] + vector[vi + blockDim.x];
    __syncthreads();

    // Step 2: Accumulate elements in the same thread block.
    for (int si = BLOCK_DIM_X / 2; si > 0; si >>= 1) {
        if (ti < si) {
            shared[ti] += shared[ti + si];
        }
        __syncthreads();
    }

    // Step 3: Move result from cache to vector.
    if (ti == 0) {
        vectorSum[blockIdx.x] = shared[0];
    }
}

int main() {
    // Step 0: Set up hyperparameters of the vector.
    int numElements = GRID_DIM_X * BLOCK_DIM_X * FACTOR;
    size_t numBytes = sizeof(int) * numElements;

    // Step 1: Init vector memories on both the cpu and the gpu.
    int *h_vector = (int*) malloc(numBytes);
    int *h_vectorSum = (int*) malloc(numBytes);
    std::fill_n(h_vector, numElements, 1);

    int *d_vector, *d_vectorSum;
    cudaMalloc(&d_vector, numBytes);
    cudaMalloc(&d_vectorSum, numBytes);

    // Step 2: Launch the kernel function to sum up the vector.
    cudaMemcpy(d_vector, h_vector, numBytes, cudaMemcpyHostToDevice);
    reduceSum<<<GRID_DIM_X, BLOCK_DIM_X>>>(d_vector, d_vectorSum);
    reduceSum<<<1, GRID_DIM_X / 2>>>(d_vectorSum, d_vectorSum);
    cudaMemcpy(h_vectorSum, d_vectorSum, numBytes, cudaMemcpyDeviceToHost);
    printf("Accumulated result is %d.\n", h_vectorSum[0]);
    // assert(h_vectorSum[0] == 65536);

    // Step 3: Clear vector memories.
    free(h_vector);
    free(h_vectorSum);
    cudaFree(d_vector);
    cudaFree(d_vectorSum);

    printf("Success!");
    return 0;
}

Accumulated result is 65536.
Success!


## Practice

In [ ]:
%%cuda

#include <algorithm>
#include <cuda_runtime.h>
#include <stdio.h>

const int FACTOR = 2;
const int GRID_DIM_X = (1 << 8) / FACTOR;
const int BLOCK_DIM_X = 1 << 8;
const int NUM_ELEMENTS = GRID_DIM_X * BLOCK_DIM_X * FACTOR;
const size_t NUM_BYTES = sizeof(int) * NUM_ELEMENTS;
const size_t NUM_BYTES_PARTIAL = sizeof(int) * GRID_DIM_X;
const size_t NUM_BYTES_SUM = sizeof(int);

__global__ void reduceSum(int *vector, int *vectorSum) {
    // Step 0: Get the vector index and the thread index.
    int ti = threadIdx.x;
    int vi = blockIdx.x * (blockDim.x * 2) + threadIdx.x;

    // Step 1: Move elements from the vector to the cache.
    __shared__ int shared[BLOCK_DIM_X];
    shared[ti] = vector[vi] + vector[vi + blockDim.x];
    __syncthreads();

    // Step 2: Accumulate vector elements.
    for (int si = BLOCK_DIM_X / 2; si > 0; si >>= 1) {
        if (ti < si) {
            shared[ti] += shared[ti + si];
        }
        __syncthreads();
    }

    // Step 3: Move results from the cache to the vector.
    if (ti == 0) {
        vectorSum[blockIdx.x] = shared[0];
    }
}

int main() {
    // Step 1: Init vector memories on both the cpu and the gpu.
    int *h_vector = (int*) malloc(NUM_BYTES);
    int *h_vectorSum = (int*) malloc(NUM_BYTES_SUM);
    std::fill_n(h_vector, NUM_ELEMENTS, 1);

    int *d_vector, *d_vectorPartialSum, *d_vectorSum;
    cudaMalloc(&d_vector, NUM_BYTES);
    cudaMalloc(&d_vectorPartialSum, NUM_BYTES_PARTIAL);
    cudaMalloc(&d_vectorSum, NUM_BYTES_SUM);

    // Step 2: Launch the kernel function to accumulate values.
    cudaMemcpy(d_vector, h_vector, NUM_BYTES, cudaMemcpyHostToDevice);
    reduceSum<<<GRID_DIM_X, BLOCK_DIM_X>>>(d_vector, d_vectorPartialSum);
    reduceSum<<<1, GRID_DIM_X / 2>>>(d_vectorPartialSum, d_vectorSum);
    cudaMemcpy(h_vectorSum, d_vectorSum, NUM_BYTES_SUM, cudaMemcpyDeviceToHost);
    printf("Accumulated result is %d.\n", *h_vectorSum);

    // Step 3: Clear vector memories.
    free(h_vector);
    free(h_vectorSum);
    cudaFree(d_vector);
    cudaFree(d_vectorPartialSum);
    cudaFree(d_vectorSum);

    printf("Success!");
    return 0;
}

Accumulated result is 65536.
Success!


# Chapter 5: CUDA Parallel Reduction Part 5

## Optimizations

1. Release idle thread warps early.

In [ ]:
%%cuda

#include <algorithm>
#include <cuda_runtime.h>
#include <stdio.h>

const int FACTOR = 2;
const int GRID_DIM_X = (1 << 8) / FACTOR;
const int BLOCK_DIM_X = 1 << 8;
const int NUM_ELEMENTS = GRID_DIM_X * BLOCK_DIM_X * FACTOR;
const size_t NUM_BYTES = sizeof(int) * NUM_ELEMENTS;
const size_t NUM_BYTES_PARTIAL = sizeof(int) * GRID_DIM_X;
const size_t NUM_BYTES_SUM = sizeof(int);

__device__ void reduceWrap(volatile int *shared, int ti) {
    shared[ti] += shared[ti + 32];
    shared[ti] += shared[ti + 16];
    shared[ti] += shared[ti + 8];
    shared[ti] += shared[ti + 4];
    shared[ti] += shared[ti + 2];
    shared[ti] += shared[ti + 1];
}

__global__ void reduceSum(int *vector, int *vectorSum) {
    // Step 0: Get the vector index and the thread index.
    int ti = threadIdx.x;
    int vi = blockIdx.x * (blockDim.x * 2) + threadIdx.x;

    // Step 1: Move elements from the vector to the cache.
    __shared__ int shared[BLOCK_DIM_X];
    shared[ti] = vector[vi] + vector[vi + blockDim.x];
    __syncthreads();

    // Step 2: Accumulate vector elements.
    for (int si = BLOCK_DIM_X / 2; si > 32; si >>= 1) {
        if (ti < si) {
            shared[ti] += shared[ti + si];
        }
        __syncthreads();
    }
    if (ti < 32) {
        reduceWrap(shared, ti);
    }

    // Step 3: Move results from the cache to the vector.
    if (ti == 0) {
        vectorSum[blockIdx.x] = shared[0];
    }
}

int main() {
    // Step 1: Init vector memories on both the cpu and the gpu.
    int *h_vector = (int*) malloc(NUM_BYTES);
    int *h_vectorSum = (int*) malloc(NUM_BYTES_SUM);
    std::fill_n(h_vector, NUM_ELEMENTS, 1);

    int *d_vector, *d_vectorPartialSum, *d_vectorSum;
    cudaMalloc(&d_vector, NUM_BYTES);
    cudaMalloc(&d_vectorPartialSum, NUM_BYTES_PARTIAL);
    cudaMalloc(&d_vectorSum, NUM_BYTES_SUM);

    // Step 2: Launch the kernel function to accumulate values.
    cudaMemcpy(d_vector, h_vector, NUM_BYTES, cudaMemcpyHostToDevice);
    reduceSum<<<GRID_DIM_X, BLOCK_DIM_X>>>(d_vector, d_vectorPartialSum);
    reduceSum<<<1, GRID_DIM_X / 2>>>(d_vectorPartialSum, d_vectorSum);
    cudaMemcpy(h_vectorSum, d_vectorSum, NUM_BYTES_SUM, cudaMemcpyDeviceToHost);
    printf("Accumulated result is %d.\n", *h_vectorSum);

    // Step 3: Clear vector memories.
    free(h_vector);
    free(h_vectorSum);
    cudaFree(d_vector);
    cudaFree(d_vectorPartialSum);
    cudaFree(d_vectorSum);

    printf("Success!");
    return 0;
}

Accumulated result is 0.
Success!


## Practice

In [ ]:
%%cuda

#include <algorithm>
#include <cuda_runtime.h>
#include <stdio.h>

const int GRID_DIM_X = (1 << 8) / 4;
const int BLOCK_DIM_X = 1 << 8;
const int NUM_ELEMENTS = GRID_DIM_X * BLOCK_DIM_X * 4;
const size_t NUM_BYTES = sizeof(int) * NUM_ELEMENTS;
const size_t NUM_BYTES_PARTIAL = sizeof(int) * GRID_DIM_X;
const size_t NUM_BYTES_SUM = sizeof(int);

__device__ void reduceWarp(volatile int *cache, int ti) {
    cache[ti] += cache[ti + 32];
    cache[ti] += cache[ti + 16];
    cache[ti] += cache[ti + 8];
    cache[ti] += cache[ti + 4];
    cache[ti] += cache[ti + 2];
    cache[ti] += cache[ti + 1];
}

__global__ void reduceSum(int *vector, int *vectorSum) {
    // Step 0: Get the vector element index and the thread index.
    int ti = threadIdx.x;
    int vi = blockIdx.x * (blockDim.x * 4) + threadIdx.x;

    // Step 1: Move elements from vector to cache.
    __shared__ int cache[BLOCK_DIM_X];
    cache[ti] = (
        vector[vi] +
        vector[vi + blockDim.x] +
        vector[vi + 2 * blockDim.x] +
        vector[vi + 3 * blockDim.x]
    );
    __syncthreads();

    // Step 2: Accumulate vector elements.
    for (int si = blockDim.x / 2; si > 32; si >>= 1) {
        if (ti < si) {
            cache[ti] += cache[ti + si];
        }
        __syncthreads();
    }
    if (ti < 32) {
        reduceWarp(cache, ti);
    }

    // Step 3: Move results from cache to vector.
    if (ti == 0) {
        vectorSum[blockIdx.x] = cache[0];
    }
}

int main() {
    // Step 1: Init vector memories on both cpu and gpu.
    int *h_vector = (int*) malloc(NUM_BYTES);
    int *h_vectorSum = (int*) malloc(NUM_BYTES_SUM);
    std::fill_n(h_vector, NUM_ELEMENTS, 1);

    int *d_vector, *d_vectorPartial, *d_vectorSum;
    cudaMalloc(&d_vector, NUM_BYTES);
    cudaMalloc(&d_vectorPartial, NUM_BYTES_PARTIAL);
    cudaMalloc(&d_vectorSum, NUM_BYTES_SUM);

    // Step 2: Launch the kernel function to accumulate all elements.
    cudaMemcpy(d_vector, h_vector, NUM_BYTES, cudaMemcpyHostToDevice);
    reduceSum<<<GRID_DIM_X, BLOCK_DIM_X>>>(d_vector, d_vectorPartial);
    reduceSum<<<1, GRID_DIM_X / 4>>>(d_vectorPartial, d_vectorSum);
    cudaMemcpy(h_vectorSum, d_vectorSum, NUM_BYTES_SUM, cudaMemcpyDeviceToHost);
    printf("Accumulated result is %d.\n", *h_vectorSum);

    // Step 3: Clear vector memories.
    free(h_vector);
    free(h_vectorSum);
    cudaFree(d_vector);
    cudaFree(d_vectorPartial);
    cudaFree(d_vectorSum);

    printf("Success!");
    return 0;
}

Accumulated result is 0.
Success!


# Chapter 6: CUDA Parallel Reduction Part 6

## Optimizations

In [45]:
%%cuda

#include <algorithm>
#include <cooperative_groups.h>
#include <cuda_runtime.h>
#include <stdio.h>

using namespace cooperative_groups;
using namespace std;

__device__ int threadSum(int *vector, int numElements) {
    int sum = 0;
    int g_ti = blockIdx.x * blockDim.x + threadIdx.x;
    int g_tc = gridDim.x * blockDim.x;

    for (int i = g_ti; i < numElements / 4; i += g_tc) {
        int4 element = ((int4*) vector)[i];
        sum += element.x + element.y + element.z + element.w;
    }

    return sum;
}

__device__ int blockSum(thread_group& g, int *cache, int sumOfThread) {
    // Step 0: Get the thread index.
    int b_ti = g.thread_rank();

    // Step 1: Move values to cache.
    cache[b_ti] = sumOfThread;
    g.sync();

    // Step 2: Accumulate values with divide-and-conquer approach.
    for (int si = g.size() / 2; si > 0; si >>= 1) {
        if (b_ti < si) {
            cache[b_ti] += cache[b_ti + si];
        }
        g.sync();
    }

    // Step 3: Return results.
    return cache[0];
}

__device__ void gridSum(thread_group& g, int *sum, int sumOfBlock) {
    if (g.thread_rank() == 0) {
        atomicAdd(sum, sumOfBlock);
    }
}

__global__ void reduceSum(int *vector, int *sum, int numElements) {
    // Step 0: Set up parameters and shared caches.
    extern __shared__ int cache[];
    auto g = this_thread_block();

    // Step 1: Sum up all elements in the vector.
    int sumOfThread = threadSum(vector, numElements);
    int sumOfBlock = blockSum(g, cache, sumOfThread);
    gridSum(g, sum, sumOfBlock);
}

int main() {
    // Step 0: Set up parameters.
    int threadFactor = 4;
    int gridDimX = 1 << 8;
    int blockDimX = (1 << 8) / threadFactor;
    int numMemElements = gridDimX * blockDimX * threadFactor;
    size_t numMemBytes = sizeof(int) * numMemElements;
    size_t numCacheBytes = sizeof(int) * blockDimX;

    // Step 1: Init memories.
    int *vector, *sum;
    cudaMallocManaged(&vector, numMemBytes);
    cudaMallocManaged(&sum, sizeof(int));
    fill_n(vector, numMemElements, 1);

    // Step 2: Accumulate vector elements.
    reduceSum<<<gridDimX, blockDimX, numCacheBytes>>>(vector, sum, numMemElements);
    cudaDeviceSynchronize();
    printf("Accumulated result is %d.\n", sum[0]);

    // Step 3: Clear memories.
    cudaFree(vector);
    cudaFree(sum);

    printf("Success!");
    return 0;
}

Accumulated result is 65536.
Success!


## Practice 1

with `<cooperative_groups.h>`

In [64]:
%%cuda

#include <assert.h>
#include <algorithm>
#include <cooperative_groups.h>
#include <stdio.h>

using namespace cooperative_groups;
using namespace std;

__device__ int threadSum(int *vector, int m_numElements) {
    int sum = 0;
    int g_ti = blockIdx.x * blockDim.x + threadIdx.x;
    int g_tc = gridDim.x * blockDim.x;

    for (int vi = g_ti; vi < m_numElements / 4; vi += g_tc) {
        int4 element4 = ((int4*) vector)[vi];
        sum += element4.x + element4.y + element4.z + element4.w;
    }

    return sum;
}

__device__ int blockSum(thread_group &group, int sumOfThread, int *cache) {
    // Step 0: Get the thread index.
    int b_ti = group.thread_rank();

    // Step 1: Move values to the cache.
    cache[b_ti] = sumOfThread;
    group.sync();

    // Step 2: Divide-and-conquer the thread sum in one block.
    for (int si = group.size() / 2; si > 0; si >>= 1) {
        if (b_ti < si) {
            cache[b_ti] += cache[b_ti + si];
        }
        group.sync();
    }

    // Step 3: Return the sum of the block.
    return cache[b_ti];
}

__device__ void gridSum(thread_group &group, int *sum, int sumOfBlock) {
    if (group.thread_rank() == 0) {
        atomicAdd(sum, sumOfBlock);
    }
}

__global__ void reduceSum(int *vector, int *sum, int m_numElements) {
    // Step 0: Set up parameters and shared cache.
    extern __shared__ int cache[];
    thread_group group = this_thread_block();

    // Step 1: Sum up elements in thread-block-grid levels.
    int sumOfThread = threadSum(vector, m_numElements);
    int sumOfBlock = blockSum(group, sumOfThread, cache);
    gridSum(group, sum, sumOfBlock);
}

int main() {
    // Step 0: Set up parameters.
    int d_numThreadTask = 4;
    int d_gridDimX = 1 << 8;
    int d_blockDimX = (1 << 8) / d_numThreadTask;
    int m_numElements = d_gridDimX * d_blockDimX * d_numThreadTask;
    size_t m_numBytes = sizeof(int) * m_numElements;
    size_t c_numBytes = sizeof(int) * d_blockDimX;

    // Step 1: Init memories.
    int *vector, *sum;
    cudaMallocManaged(&vector, m_numBytes);
    cudaMallocManaged(&sum, sizeof(int));
    fill_n(vector, m_numElements, 1);

    printf("vector[0] = %d\n", vector[0]);

    // Step 2: Launch the kernel function to aggregate numbers.
    reduceSum<<<d_gridDimX, d_blockDimX, c_numBytes>>>(vector, sum, m_numElements);
    cudaDeviceSynchronize();

    printf("Expected: *sum = 65536\n");
    printf("Actual: *sum = %d\n", *sum);
    assert(*sum == 1 << 16);

    // Step 3: Clear memories.
    cudaFree(vector);
    cudaFree(sum);

    printf("Success!");
    return 0;
}

vector[0] = 1
Expected: *sum = 65536
Actual: *sum = 65536
Success!


## Practice 2

without `<cooperative_groups.h>`

In [69]:
%%cuda

#include <assert.h>
#include <algorithm>
#include <stdio.h>

using namespace std;

__device__ int threadSum(int *vector, int m_numElements) {
    int sum = 0;
    int g_ti = blockIdx.x * blockDim.x + threadIdx.x;
    int g_tc = gridDim.x * blockDim.x;

    for (int vi = g_ti; vi < m_numElements / 4; vi += g_tc) {
        int4 element4 = ((int4*) vector)[vi];
        sum += element4.x + element4.y + element4.z + element4.w;
    }

    return sum;
}

__device__ int blockSum(int sumOfThread, int *cache) {
    // Step 0: Get the thread index.
    int b_ti = threadIdx.x;

    // Step 1: Move values to the cache.
    cache[b_ti] = sumOfThread;
    __syncthreads();

    // Step 2: Divide-and-conquer the thread sum in one block.
    for (int si = blockDim.x / 2; si > 0; si >>= 1) {
        if (b_ti < si) {
            cache[b_ti] += cache[b_ti + si];
        }
        __syncthreads();
    }

    // Step 3: Return the sum of the block.
    return cache[b_ti];
}

__device__ void gridSum(int *sum, int sumOfBlock) {
    if (threadIdx.x == 0) {
        atomicAdd(sum, sumOfBlock);
    }
}

__global__ void reduceSum(int *vector, int *sum, int m_numElements) {
    // Step 0: Set up parameters and shared cache.
    extern __shared__ int cache[];

    // Step 1: Sum up elements in thread-block-grid levels.
    int sumOfThread = threadSum(vector, m_numElements);
    int sumOfBlock = blockSum(sumOfThread, cache);
    gridSum(sum, sumOfBlock);
}

int main() {
    // Step 0: Set up parameters.
    int d_numThreadTask = 4;
    int d_gridDimX = 1 << 8;
    int d_blockDimX = (1 << 8) / d_numThreadTask;
    int m_numElements = d_gridDimX * d_blockDimX * d_numThreadTask;
    size_t m_numBytes = sizeof(int) * m_numElements;
    size_t c_numBytes = sizeof(int) * d_blockDimX;

    // Step 1: Init memories.
    int *vector, *sum;
    cudaMallocManaged(&vector, m_numBytes);
    cudaMallocManaged(&sum, sizeof(int));
    fill_n(vector, m_numElements, 1);

    printf("vector[0] = %d\n", vector[0]);

    // Step 2: Launch the kernel function to aggregate numbers.
    reduceSum<<<d_gridDimX, d_blockDimX, c_numBytes>>>(vector, sum, m_numElements);
    cudaDeviceSynchronize();

    printf("Expected: *sum = 65536\n");
    printf("Actual: *sum = %d\n", *sum);
    assert(*sum == 1 << 16);

    // Step 3: Clear memories.
    cudaFree(vector);
    cudaFree(sum);

    printf("Success!");
    return 0;
}

vector[0] = 1
Expected: *sum = 65536
Actual: *sum = 65536
Success!
